## 05_Modeling.ipynb
###  **🎯 Objective:**

The objective of this notebook is to define, compile, and train a Convolutional Neural Network (CNN) model to detect powdery mildew in cherry leaf images. This phase follows the preprocessing stage and forms the core of the modeling step in the CRISP-DM methodology.



## Model Architecture
We define a deep learning model using TensorFlow Keras. The model consists of:
 - Convolutional layers for feature extraction
 - MaxPooling layers to reduce spatial dimensions
 - GlobalAveragePooling to reduce the feature map
 - Dense layers for decision making
 - Dropout layer for regularization
 - Sigmoid activation for binary classification

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json, pickle

2025-06-04 17:37:00.936098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# === Model Definition ===
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Dense(1, activation='sigmoid')
    Dense(2, activation='softmax') 
])

/Users/dappiah/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Compile the Model
We compile the model using:
 - Loss function: binary_crossentropy (for binary classification)
 - Optimizer: adam
 - Evaluation metric: accuracy

In [3]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

 ### Load Train and Validation Data

### Load Preprocessed Data
Using ImageDataGenerator, we load:
 - Augmented training images
 - Rescaled validation images

This step prepares the model to be trained on realistic and varied data, improving generalization.

In [4]:
# === Image Augmentation ===
img_shape = (256, 256)
batch_size = 32

train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

test_aug = ImageDataGenerator(rescale=1./255)

In [5]:
# === Data Loaders with class_mode='categorical' ===
train_data = train_aug.flow_from_directory(
    "../inputs/split-leaves/train",
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='categorical'  # <-- Updated from 'binary'
)

val_data = test_aug.flow_from_directory(
    "../inputs/split-leaves/validation",
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='categorical',  # <-- Updated from 'binary'
    shuffle=False
)


Found 2944 images belonging to 2 classes.
Found 840 images belonging to 2 classes.


In [6]:
# === Summary & Training ===
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,018 (429.76 KB)

 Trainable params: 110,018 (429.76 KB)

 Non-trainable params: 0 (0.00 B)

### Train the Model and Save Artifacts

### Training the Model
We train the model using model.fit(...) with:
 - EarlyStopping to prevent overfitting
 - ModelCheckpoint to save the best-performing model based on validation loss
 - We run the model for up to 20 epochs, monitoring both training and validation accuracy/loss.

In [ ]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("../outputs/mildew_model_softmax.keras", save_best_only=True)
]

history = model.fit(
    train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=callbacks
)

/Users/dappiah/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.5577 - loss: 0.6705 - val_accuracy: 0.9690 - val_loss: 0.2332
Epoch 2/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.9713 - loss: 0.1367 - val_accuracy: 0.9881 - val_loss: 0.0835
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.9887 - loss: 0.0600 - val_accuracy: 0.9845 - val_loss: 0.0808
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 0.9884 - loss: 0.0447 - val_accuracy: 0.9833 - val_loss: 0.0653
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.9939 - loss: 0.0213 - val_accuracy: 0.9929 - val_loss: 0.0206
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.9983 - loss: 0.0093 - val_accuracy: 0.9810 - val_loss: 0.0561
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.9770 - loss: 0.0598 - val_accuracy: 0.9917 - val_loss: 0.0195
Epoch 8/20
12/92 ━━━━━━━━━━━━━━━━━━━━ 1:39 1s/step - accuracy: 0.9975 - loss: 0.0276

### Save Model and History
We save the trained model to outputs/mildew_model.h5 and store the training history in:
 - training_history.pkl (binary format)
 - history.json (readable format)

This allows us to later visualize training progress without re-training the model.

In [ ]:
# Save training history
import json, pickle

# with open("../outputs/training_history.pkl", "wb") as f:
with open("../outputs/training_history_softmax.pkl", "wb") as f:
    pickle.dump(history.history, f)

# with open("../outputs/history.json", "w") as f:
with open("../outputs/history_softmax.json", "w") as f:
    json.dump(history.history, f)


#### To avoid retraining every time, comment out model.fit(...) and just use this:

In [ ]:
# with open("../outputs/training_history.pkl", "rb") as f:
#    history_data = pickle.load(f)